In [1]:
import random
import json

In [2]:
def list_to_string(state):
    string=''
    for i in state:
        string += i
    return string

In [30]:
def player_turn(state):
    num = state.count('-')
    if num % 2 == 0:
        return 'O'
    else:
        return 'X'

In [4]:
def is_terminal(state):
    if state[0] == state[1] == state[2] == 'X':
        return True, 1
    elif state[3] == state[4] == state[5] == 'X':
        return True, 1
    elif state[6] == state[7] == state[8] == 'X':
        return True, 1
    elif state[0] == state[3] == state[6] == 'X':
        return True, 1
    elif state[1] == state[4] == state[7] == 'X':
        return True, 1
    elif state[2] == state[5] == state[8] == 'X':
        return True, 1
    elif state[0] == state[4] == state[8] == 'X':
        return True, 1
    elif state[2] == state[4] == state[6] == 'X':
        return True, 1
    elif state[0] == state[1] == state[2] == 'O':
        return True, -1
    elif state[3] == state[4] == state[5] == 'O':
        return True, -1
    elif state[6] == state[7] == state[8] == 'O':
        return True, -1
    elif state[0] == state[3] == state[6] == 'O':
        return True, -1
    elif state[1] == state[4] == state[7] == 'O':
        return True, -1
    elif state[2] == state[5] == state[8] == 'O':
        return True, -1
    elif state[0] == state[4] == state[8] == 'O':
        return True, -1
    elif state[2] == state[4] == state[6] == 'O':
        return True, -1
    elif state.count('-') == 0:
        return True, 0
    else:
        return False, 0

In [31]:
tree = {}
terminal = []
child_parent = {}
Scores = {}
def create_tree(state = ['-']*9):
    global tree
    string_of_state = list_to_string(state)
    tree[string_of_state] = []
    turn = player_turn(state)

    if not is_terminal(state)[0]:

        for i in range(9):
            state_copy = state.copy()
            if state[i] == '-':
                state_copy[i] = turn
                state_copy_string = list_to_string(state_copy)
                tree[string_of_state].append(state_copy_string)

                if state_copy_string not in child_parent.keys():
                    child_parent[state_copy_string] = []
                if string_of_state not in child_parent[state_copy_string]:
                    child_parent[state_copy_string].append(string_of_state)


                Scores[state_copy_string] = is_terminal(state_copy_string)[1]

                create_tree(state_copy)
    else:
        if string_of_state not in terminal:
            terminal.append(string_of_state)

create_tree()
child_parent['---------']=[None]
Scores['---------'] = 0

In [41]:
alpha = 0.3104
def update_helper(i, t1 = 0):
    global alpha
    t1 = Scores[i]
    if i != '---------':
        parents = child_parent[i]
        for j in parents:
            t2 = Scores[j]
            value = t2 + alpha*(t1 - t2)
            Scores[j] = value
#             t1 = value
            update_helper(j, t1)


def update_value():
    for i in terminal*100:
        update_helper(i)
    for k in range(10000):
        num = random.randrange(958)
        update_helper(terminal[num])

update_value()


In [43]:
data_to_store = {'dict1':tree, 'dict3':Scores}

json_data = json.dumps(data_to_store, indent=2)
with open('Reinforcement_data.json', 'w') as json_file:
    json_file.write(json_data)

In [38]:
def position_for_move(state, action):
    for i in range(9):
        if state[i] == '-' and action[i] != '-':
            return i

def computers_move(state, player):
    state = list_to_string(state)
    childrens = tree[state]
    score_dict = {}

    for i in childrens:
        score_dict[Scores[i]] = i

    value = True
    lst = [1,3,5,7,9]
    while value:
        num = random.randrange(5)
        if player == lst[num]:
            score = max(score_dict.keys())
        else:
            score = min(score_dict.keys())
        value = position_for_move(state, score_dict[score])
        if value is None:
            del score_dict[score]
        else:
            return value

In [39]:
def play_game():
    state = ['-']*9
    display(state)
    player = 0
    while not is_terminal(state)[0]:
        if player % 2 != 0:
            index = int(input("Enter position (0, 8):- "))
            state[index] = 'X'
        else:
            index = computers_move(state, player)
            state[index]='O'

        display(state)
        player += 1
        terminal = is_terminal(state)

        if terminal[1] == -1:
            return 'Sorry!! You lost'
        elif terminal[1] == 1:
            return 'Congrats You won'
        elif terminal[0] and terminal[1] == 0:
            return 'Match draw'

In [40]:
play_game()

-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
-------------------
|  O  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
Enter position (0, 8):- 1
-------------------
|  O  |  X  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
-------------------
|  O  |  X  |  O  |
-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
Enter position (0, 8):- 4
-------------------
|  O  |  X  |  O  |
-------------------
|  -  |  X  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
-------------------
|  O  |  X  |  O  |
-------------------
|  -  |  X  |  -  |
-------------------
|  -  |  -  |  O  |
-------------------
Enter position (0, 8):- 7
-------------------
|  O  |  X  |  O  |
-------------------
|  -  |  X  |  -  |
--

'Congrats You won'